# Nave Sentiment Movie Corpus (NSMC) 네이버 영화 리뷰 분석기

In [1]:
!pip install konlpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 19.4 MB 786 kB/s 
     |████████████████████████████████| 453 kB 61.3 MB/s 


In [2]:
from gensim.models import Word2Vec
from konlpy.tag import Komoran
import time

In [3]:
# 네이버 영화 리뷰 데이터 읽어옴
def read_review_data(filename):
  with open(filename, 'r') as f:
    data = [line.split('\t') for line in f.read().splitlines()]
    data = data[1:] # 헤더 제거
  return data

In [4]:
# 학습 시간 측정 시작
start = time.time()

In [6]:
# 리뷰 파일 읽어오기
print('1) 말뭉치 데이터 읽기 시작')
review_data = read_review_data('ratings.txt')
print(len(review_data)) # 리뷰 데이터 전체 개수
print('1) 말뭉치 데이터 읽기 완료: ',time.time() - start)

1) 말뭉치 데이터 읽기 시작
76302
1) 말뭉치 데이터 읽기 완료:  14.04204773902893


In [8]:
# 문장 단위로 명사만 추출해 학습 입력 데이터로 만듦
print('2) 형태소에서 명사만 추출 시작')
komoran = Komoran()
docs = [komoran.nouns(sentence[1]) for sentence in review_data]
print('2) 형태소에서 명사만 추출 완료: ', time.time() - start)

2) 형태소에서 명사만 추출 시작
2) 형태소에서 명사만 추출 완료:  76.42251467704773


**### Word2Vec 모델의 주요 하이퍼파라미터(Hyperparameter)**

*   **sentecences**: Word2Vec 모델 학습에 필요한 문장 데이터. Word2Vec 모델의 입력값으로 사용
*   **size** : 단어 임베딩 벡터의 차원(크기)
*   **window** : 주변 단어 윈도우의 크기
*   **hs**: 1 (모델 학습에 softmax 사용), 0 (negative 옵션값이 0이 아닐때 음수 샘플링으로 사용)
*   **min_count** : 단어 최소 빈도 수 제한(설정된 min_count 빈도 수 이하의 단어들은 학습하지 않음)
*   **sg**: 0 (CBOW 모델), 1 (skip-gram 모델)


In [12]:
# Word2Vec 모델 학습
print('3) Word2Vec 모델 학습 시작')
model = Word2Vec(sentences=docs, size=200, window=4, hs=1, min_count=2, sg=1)
print('3) Word2Vec 모델 학습 완료 : ', time.time() - start)

3) Word2Vec 모델 학습 시작
3) Word2Vec 모델 학습 완료 :  141.76306223869324


In [13]:
# 모델 저장
print( '4 ) 학습된 모델 저장 시작')
model.save('nvmc.model')
print('4) 학습된 모델 저장 완료 : ', time.time() - start)

4 ) 학습된 모델 저장 시작
4) 학습된 모델 저장 완료 :  144.04411387443542


In [14]:
# 학습된 말뭉치 수, 코퍼스 내 전체 단어 수
print("corpus_count : ", model.corpus_count)
print("corpus_total_words : ", model.corpus_total_words)

corpus_count :  76302
corpus_total_words :  394675


# Word2Vec 모델 활용

In [15]:
from gensim.models import Word2Vec

In [16]:
# 모델 로딩
model = Word2Vec.load('nvmc.model')
print("corpus_total_words : ", model.corpus_total_words)

corpus_total_words :  394675


In [17]:
# '니노'라는 단어로 생성한 단어 임베딩 벡터
print('니노 : ', model.wv['니노'])

니노 :  [-0.00149873 -0.04908129  0.00935159  0.02165198  0.01314906 -0.07479876
 -0.05846514 -0.03072808 -0.05268116  0.04051319  0.04908348 -0.00075928
  0.01513309 -0.00853626 -0.00911729  0.01587292 -0.01241465 -0.04944022
 -0.00692094 -0.06702016  0.05484813 -0.00727925 -0.0158548  -0.01658519
 -0.00556976  0.08253369  0.03857873  0.0555262   0.02279759  0.02991422
 -0.01262016  0.02972154  0.05672603 -0.03761302 -0.01326295  0.04687921
  0.03646654  0.00548177  0.10230421 -0.01135169  0.05767523  0.09234758
  0.0076671   0.08284823  0.06868663 -0.01779663  0.03360683  0.00489422
 -0.0822499   0.00602328 -0.06919399 -0.00290626 -0.00863019 -0.01238457
  0.01173775  0.04384543 -0.02025903 -0.02905651  0.05718726  0.02144219
  0.03865674  0.04217994  0.0194597  -0.01745635  0.01887674 -0.006362
 -0.03527401 -0.02155095  0.02603192 -0.00165662 -0.02844302  0.00709218
 -0.07819697  0.00544401 -0.03030618 -0.00418158  0.01570269  0.03445189
 -0.03725921 -0.02273048 -0.0274572  -0.0026730

In [18]:
# '아라시'라는 단어로 생성한 단어 임베딩 벡터
print('아라시 : ', model.wv['아라시'])

아라시 :  [-5.5138603e-02 -2.9063331e-02 -1.9433297e-02 -7.9363398e-03
  7.5453669e-02 -4.7374260e-02 -6.1470244e-02 -5.4266453e-03
 -3.6374446e-02  5.0549731e-03  8.2089528e-02  1.9794611e-02
  1.3277784e-02 -1.4622936e-02  1.8648284e-02  3.3636518e-02
  1.9068308e-02 -2.5589399e-02  4.4785254e-02 -5.8911528e-02
  5.0670333e-02 -4.5091420e-02  3.3457885e-03 -2.8578471e-02
  2.2017728e-03  5.8973953e-02  1.9732762e-02  6.7933932e-02
 -5.1985928e-03  1.4101630e-02  9.5692789e-04 -1.1693462e-02
  4.6851486e-03 -7.5139478e-03 -1.4172492e-02  5.9395257e-02
  5.5360861e-02 -2.0993019e-02  1.6675156e-02 -1.3726545e-02
  7.5392038e-02  2.1643789e-02  5.6337815e-02  3.5573125e-02
  3.0113185e-02 -4.3458506e-03  2.0941390e-02  2.4790481e-02
 -9.6672349e-02 -1.1183744e-02 -6.0834784e-02 -2.6052225e-02
  8.0831855e-04 -4.8809582e-03 -1.3431071e-02  2.0959087e-02
 -5.5240747e-02 -2.6277393e-02  7.3155038e-02 -2.9809469e-02
  5.6338042e-02  5.2032880e-02  2.5175380e-02 -2.8589644e-02
 -2.4779886e-02 -

In [25]:
# 단어 유사도 계산
print("아라시 = 니노\t", model.wv.similarity(w1='아라시', w2='니노'))
print("아라시 = 소녀시대\t", model.wv.similarity(w1='아라시', w2='소녀시대'))
print("일본 = 아라시\t", model.wv.similarity(w1='일본', w2='아라시'))
print("일본 = 니노\t", model.wv.similarity(w1='일본', w2='니노'))

아라시 = 니노	 0.7062562
아라시 = 소녀시대	 0.37712428
일본 = 아라시	 0.36570537
일본 = 니노	 0.5565038


In [36]:
# 가장 유사한 단어 추출
print(model.wv.most_similar("아라시", topn=5))
print(model.wv.most_similar("니노", topn=5))
print(model.wv.most_similar("일본", topn=5))
print(model.wv.most_similar("오노", topn=5))

[('김광석', 0.8839106559753418), ('라유', 0.8655327558517456), ('재주', 0.8648266196250916), ('파타고니아', 0.8593234419822693), ('매도', 0.8577490448951721)]
[('출간', 0.8835716247558594), ('도사', 0.8755406737327576), ('개인주의', 0.8740397691726685), ('인신매매', 0.8710757493972778), ('침몰', 0.8685587644577026)]
[('프랑스', 0.7398494482040405), ('판', 0.6987419128417969), ('영국', 0.6857080459594727), ('실사', 0.6768336296081543), ('일본인', 0.6733152866363525)]
[('낮술', 0.8794459104537964), ('탄광', 0.8629952669143677), ('주유', 0.8608558177947998), ('리포트', 0.8588311672210693), ('날개', 0.858605146408081)]
